## 📌 Objective
Create a simple system that can:
1. Accept PDF documents from users
2. Understand questions about the document
3. Provide accurate answers using AI
4. Automatically improve its answers by refining questions

## ❓ Problem Statement
Traditional document systems struggle with:
- Handling large documents
- Understanding natural language questions
- Providing precise answers from long texts
- Improving answers automatically

## 💡 Solution
Our system solves these problems using:
1. **Document Processing**: Breaks files into manageable chunks
2. **Smart Storage**: Stores content for quick searching
3. **AI Assistance**: Uses language models to understand questions
4. **Self-Improvement**: Automatically refines questions for better answers

## 🤖 What is Agentic RAG?
**RAG (Retrieval-Augmented Generation)**:
- Combines document search with AI answers
- "Looks up" information before answering

**Agentic RAG** adds:
- Ability to automatically improve questions
- Self-correcting answers
- Better understanding through context

*Example:*
If you ask "What is the Name of the Customer", the system will change it to "What is the name of the customer referenced in the Master Agreement and Service Order?" for better results

---

In [2]:
# Step 1: Install required libraries
!pip install chromadb langchain pypdf2 sentence-transformers pyboxen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 18.0 MB/s eta

## 📦 **Import Statements & Their Purpose**
```python
import ipywidgets as widgets  # For creating interactive UI elements
from IPython.display import display, clear_output  # To control UI output
from langchain.vectorstores import Chroma  # For vector-based search
from langchain.embeddings import HuggingFaceEmbeddings  # Convert text into AI-readable format
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Break text into small pieces
from langchain.llms import OpenAI  # AI-powered language model
from langchain.prompts import PromptTemplate  # Helps structure prompts
from PyPDF2 import PdfReader  # Read and extract text from PDF files
import chromadb  # The ChromaDB client for database interaction
from pyboxen import boxen  # Stylish text boxes for output
import os  # Access environment variables (e.g., API keys)
```

### 🌐 **About LangChain (In Simple Terms)**
LangChain is a framework that helps developers **connect AI models with external data sources** like databases or APIs. In this project, we use LangChain to:
- Embed text for efficient search.
- Retrieve relevant information from the database.
- Generate answers using OpenAI’s model.


In [5]:
! pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [6]:
# Step 2: Import necessary modules
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from PyPDF2 import PdfReader
import chromadb
from pyboxen import boxen
import os

### 📌 **Step 3: Initializing Components**
```python
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
chroma_client = chromadb.PersistentClient(path="./chroma_db")
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
llm = OpenAI(temperature=0)
```
- **Embeddings:** We use `HuggingFaceEmbeddings` to convert text into numerical format.
- **ChromaDB Client:** Creates a database at `./chroma_db` to store text embeddings.
- **OpenAI API:** We set the API key to access OpenAI’s language model.
[Generate Your OpenAI API Key](https://github.com/initmahesh/MLAI-community-labs/tree/main/Class-Labs/Lab-0(Pre-requisites))

- **LLM Initialization:** We set the temperature to `0` for more deterministic responses.

In [30]:
# Step 3: Initialize components
# Initialize embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Initialize LLM (Replace with your API key)
os.environ["OPENAI_API_KEY"] = ""
llm = OpenAI(temperature=0)

ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

#### **Processing the Uploaded File**

- Reads the PDF and extracts text.  
- Splits the text into smaller **chunks**.  
- Stores these chunks in **ChromaDB**.  

We create a file upload widget to allow users to upload a PDF.  

[📄 **Reference Document You Can Use**](https://drive.google.com/file/d/1WWa_TgI49HIAGFuXTNvMLtkFBU6ZduHq/view?usp=sharing)  

- When you run the below cell, you will see an **upload button**. Click on it, upload your document (you can use the reference document provided), and then click **Process File**.  
- You will get a confirmation message: **"File processed and stored in ChromaDB!"**  


In [26]:
# Step 4: File upload widget
from io import BytesIO
uploader = widgets.FileUpload(accept='.pdf', multiple=False)
display(uploader)

process_btn = widgets.Button(description="Process File")
process_output = widgets.Output()

def process_file(b):
    with process_output:
        clear_output()
        if not uploader.value:
            print("No file uploaded!")
            return

        # Get the first uploaded file's content
        # The 'uploader.value' now contains a dictionary, and you need to iterate
        # through its values (which are the actual file data objects)
        for filename, file_info in uploader.value.items():
            # Assuming there's only one file due to 'multiple=False' in the uploader
            pdf = PdfReader(BytesIO(file_info['content']))  # Read using BytesIO
            break  # Exit loop as we've processed the only file

        text = "\n".join([page.extract_text() for page in pdf.pages])

        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=100
        )
        chunks = text_splitter.split_text(text)

        # Create Chroma collection
        Chroma.from_texts(
            chunks, embeddings,
            client=chroma_client,
            collection_name="doc_collection"
        )
        print(boxen("File processed and stored in ChromaDB!", title="Success", color="green"))

display(process_btn, process_output)
process_btn.on_click(process_file)

FileUpload(value={}, accept='.pdf', description='Upload')

Button(description='Process File', style=ButtonStyle())

Output()

### 📌 **Step 5: Query Processing Functions**

#### **Retrieve Relevant Chunks**
```python
def retrieve_chunks(query, collection_name="doc_collection"):
    collection = chroma_client.get_collection(collection_name)
    results = collection.query(query_texts=[query], n_results=3)
    return results['documents'][0]
```
- Searches for the most relevant text chunks **based on the user’s query**.

#### **Generate AI-Powered Answers**
```python
def generate_answer(query, chunks):
    context = "\n\n".join(chunks)
    prompt = f"Answer this query: {query}\nUsing this context:\n{context}"
    return llm(prompt)
```
- Uses OpenAI’s language model to generate an answer based on **retrieved chunks**.

#### **Optimize Query for Better Results**
```python
def transform_query(original_query, original_chunks):
    context = "\n".join(original_chunks[:2])
    template = """Optimize this query for better results. Consider context:
    Context: {context}
    Original Query: {query}
    Optimized Query:"""
    return llm(template.format(context=context, query=original_query))
```
- Uses AI to **rewrite the query** for improved results.

---

In [27]:
# Step 5: Query processing functions
def retrieve_chunks(query, collection_name="doc_collection"):
    collection = chroma_client.get_collection(collection_name)
    results = collection.query(query_texts=[query], n_results=3)
    return results['documents'][0]

def generate_answer(query, chunks):
    context = "\n\n".join(chunks)
    prompt = f"Answer this query: {query}\nUsing this context:\n{context}"
    return llm(prompt)

def transform_query(original_query, original_chunks):
    context = "\n".join(original_chunks[:2])  # Use first 2 chunks for context
    template = """Optimize this query for better results. Consider context:
    Context: {context}
    Original Query: {query}
    Optimized Query:"""
    return llm(template.format(context=context, query=original_query))

# 📌 Step 5.5: Add Knowledge Graph Component

## 🧠 What is the Knowledge Graph Component?
The **Knowledge Graph Component** enhances user queries by mapping common terms to predefined business relationships. This ensures queries align with standardized business terminology, improving response accuracy.

## ⚙️ How It Works:
1. **Predefined Mappings:**  
   The system maintains a dictionary of business term relationships:
   ```python
   kg_mappings = {
       "customer": "customer referenced in the Master Agreement and Service Order",
       "Master Agreement": "Contract",
       "Service Order": "Service Agreement",
       "client": "Organization",
       "product": "Solution"
   }


In [28]:
# Step 5.5: Add Knowledge Graph Component
def knowledge_graph(query):
    """Enhance queries using predefined business relationships"""
    # Define dummy knowledge graph relationships
    kg_mappings = {
        "customer": "customer referenced in the Master Agreement and Service Order",
        "Master Agreement": "Contract",
        "Service Order": "Service Agreement",
        "client": "Organization",
        "product": "Solution"
    }

    # Create transformation prompt
    prompt = f"""Based on these business term mappings: {kg_mappings}
    Rewrite this query using proper business terminology:
    Original Query: {query}
    Enhanced Query:"""

    return llm(prompt).strip()

### 📌 **Step 6: Query Interface**

- Creates a simple **text input** for user queries.
- Displays responses interactively.
- Retrieves an answer **before and after query optimization**.
- Displays the **difference in responses**.
- When you run this cell, you will get an input field. Enter your query (e.g., "What is the name of the customer?"), click on submit, and wait for a few seconds.
You will see:
  - The difference between the query you asked and the response you got.
  - The query generated by the LLM and the accurate response it retrieved.
  - The document chunks that both responses retrieved.

---

In [29]:
# Step 6: Query interface and knowledge graph transformation
query_input = widgets.Text(placeholder="Enter your query")
submit_btn = widgets.Button(description="Submit")
query_output = widgets.Output()

# Step 6: Updated Query Handling with Knowledge Graph
def handle_query(b):
    with query_output:
        clear_output()
        query = query_input.value

        # Initial processing
        original_chunks = retrieve_chunks(query)
        original_answer = generate_answer(query, original_chunks)

        # First-stage transformation
        transformed_query = transform_query(query, original_chunks)
        transformed_chunks = retrieve_chunks(transformed_query)
        transformed_answer = generate_answer(transformed_query, transformed_chunks)

        # Knowledge Graph enhancement
        kg_query = knowledge_graph(transformed_query)
        kg_chunks = retrieve_chunks(kg_query)
        kg_answer = generate_answer(kg_query, kg_chunks)

        # Display results
        print(boxen(
            f"ORIGINAL QUESTION: {query}\nANSWER: {original_answer}",
            title="First Try", color="red"
        ))

        print(boxen(
            f"IMPROVED QUESTION: {transformed_query}\nANSWER: {transformed_answer}",
            title="Optimized Version", color="yellow"
        ))

        print(boxen(
            f"BUSINESS TERMS QUESTION: {kg_query}\nANSWER: {kg_answer}",
            title="Professional Version", color="green"
        ))

        # Show evolution
        evolution = f"""Query Transformation Journey:
        1. Started with: {query}
        2. Context-optimized to: {transformed_query}
        3. Business-enhanced to: {kg_query}

        Final answer length: {len(kg_answer)} characters (vs original {len(original_answer)})"""

        print(boxen(evolution, title="Transformation Progress", color="blue"))

display(query_input, submit_btn, query_output)
submit_btn.on_click(handle_query)

Text(value='', placeholder='Enter your query')

Button(description='Submit', style=ButtonStyle())

Output()